<a href="https://colab.research.google.com/github/salmenhsairi/EndOfStudiesProjectNotebooks/blob/main/Layoutlmv3UBIAI_FineTuning%26Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount drive fs

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Installing requirements

In [2]:
! pip install -q git+https://github.com/huggingface/transformers.git
! pip install -q git+https://github.com/huggingface/datasets.git "dill<0.3.5" seqeval

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 2.9 MB/s 
     |████████████████████████████████| 6.6 MB 27.1 MB/s 
     |████████████████████████████████| 596 kB 67.9 MB/s 
     |████████████████████████████████| 346 kB 5.1 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 212 kB 63.9 MB/s 
     |████████████████████████████████| 140 kB 65.5 MB/s 
     |████████████████████████████████| 1.1 MB 62.9 MB/s 
     |████████████████████████████████| 86 kB 7.2 MB/s 
     |████████████████████████████████| 127 kB 59.1 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
     |████████████████████████████████| 271 kB 71.9 MB/s 
     |████████████████████████████████| 144 kB 69.0 MB/s 
     |████████████████████████████████| 112 kB 72.9 MB/s 
ERROR: pip's dependency resolver does not currently take into account a

## Pulling preprocessing file

In [3]:
! rm -r layoutlmv3FineTuning
! git clone -b master https://github.com/salmenhsairi/layoutlmv3FineTuning.git

rm: cannot remove 'layoutlmv3FineTuning': No such file or directory
Cloning into 'layoutlmv3FineTuning'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 3), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), done.


## Loading UBIAI dataset from Drive

In [4]:
#!/bin/bash
IOB_DATA_PATH = "/content/drive/MyDrive/NERDatasets/OCR_Invoice_Extraction_Tp4PlPQ.zip"
# IOB_DATA_PATH = "/content/drive/MyDrive/NERDatasets/EPIC_Case_Times_Extraction_ddvqdMl.zip"
! cd /content/
! rm -r data
! mkdir data
! cp "$IOB_DATA_PATH" data/dataset.zip 
! cd data && unzip -q dataset && rm dataset.zip
! cd ..

rm: cannot remove 'data': No such file or directory


## defining preprocessing params and running the script

In [5]:
#!/bin/bash
#preprocessing args
TEST_SIZE = 0.2
DATA_OUTPUT_PATH = "/content/"

In [6]:
! python3 layoutlmv3FineTuning/preprocess.py --valid_size $TEST_SIZE --output_path $DATA_OUTPUT_PATH

100% 1/1 [00:02<00:00,  2.17s/ba]
Downloading: 100% 275/275 [00:00<00:00, 391kB/s]
Downloading: 100% 1.12k/1.12k [00:00<00:00, 1.45MB/s]
Downloading: 100% 856/856 [00:00<00:00, 1.27MB/s]
Downloading: 100% 878k/878k [00:00<00:00, 7.67MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 5.82MB/s]
100% 1/1 [00:03<00:00,  3.01s/ba]
100% 1/1 [00:00<00:00,  1.21ba/s]
Flattening the indices: 100% 1/1 [00:00<00:00, 20.65ba/s]
Flattening the indices: 100% 1/1 [00:00<00:00, 83.11ba/s]


## Proceed with Training

In [1]:
from datasets import load_metric
from transformers import TrainingArguments, Trainer
from transformers import LayoutLMv3ForTokenClassification,AutoProcessor
from transformers.data.data_collator import default_data_collator

In [2]:
# load datasets
from datasets import load_from_disk
train_dataset = load_from_disk(f'/content/train_split')
eval_dataset = load_from_disk(f'/content/eval_split')

In [3]:
labels = train_dataset.features["labels"].feature.names
num_labels = len(labels)
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label

In [4]:
metric = load_metric("seqeval")
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels,zero_division='0')
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

## loading model and preprocessor (also required for Hugging face trainer)

In [5]:
label_list = train_dataset.features["labels"].feature.names
num_labels = len(label_list)
label2id, id2label = dict(), dict()
for i, label in enumerate(label_list):
    label2id[label] = i
    id2label[i] = label

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## let's train the model

In [6]:
NUM_TRAIN_EPOCHS = 6 
PER_DEVICE_TRAIN_BATCH_SIZE = 1
PER_DEVICE_EVAL_BATCH_SIZE = 1
LEARNING_RATE = 4e-5

In [7]:
training_args = TrainingArguments(output_dir="test",
                                  max_steps=1500,
                                  # num_train_epochs=NUM_TRAIN_EPOCHS,
                                  logging_strategy="epoch",
                                  save_total_limit=1,
                                  per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
                                  per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH_SIZE,
                                  learning_rate=LEARNING_RATE,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  # eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1")

In [8]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,1.131600,0.770097,1.000000,0.000000,0.000000,0.886818
2,0.686600,0.735390,1.000000,0.000000,0.000000,0.886818
3,0.619900,0.666163,1.000000,0.000000,0.000000,0.886818
4,0.543000,0.614050,0.281250,0.092784,0.139535,0.889481
5,0.463300,0.573233,0.228916,0.195876,0.211111,0.890812
6,0.409500,0.497680,0.228261,0.216495,0.222222,0.895473
7,0.335000,0.500194,0.195312,0.257732,0.222222,0.895473
8,0.297600,0.475257,0.316239,0.381443,0.345794,0.910786
9,0.251600,0.425416,0.400000,0.536082,0.458150,0.922770
10,0.225700,0.411927,0.500000,0.597938,0.544601,0.935419


***** Running Evaluation *****
  Num examples = 11
  Batch size = 1
Saving model checkpoint to test/checkpoint-200
Configuration saved in test/checkpoint-200/config.json
Model weights saved in test/checkpoint-200/pytorch_model.bin
Feature extractor saved in test/checkpoint-200/preprocessor_config.json
tokenizer config file saved in test/checkpoint-200/tokenizer_config.json
Special tokens file saved in test/checkpoint-200/special_tokens_map.json
Deleting older checkpoint [test/checkpoint-160] due to args.save_total_limit
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:811: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning
***** Running Evaluation *****
  Num examples = 11
  Batch size = 1
Saving model checkpoint to test/checkpoint-240
Configuration saved in test/checkpoint-240/config.json
Model weights saved in test/checkpoin

evaluate the model

In [ ]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 80
  Batch size = 1
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:811: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


{'epoch': 6.0,
 'eval_accuracy': 0.9939814284076579,
 'eval_f1': 0.849673202614379,
 'eval_loss': 0.026033718138933182,
 'eval_precision': 0.8158995815899581,
 'eval_recall': 0.8863636363636364,
 'eval_runtime': 8.7292,
 'eval_samples_per_second': 9.165,
 'eval_steps_per_second': 9.165}

## save the model for upcoming fine-tuning/infrence

In [ ]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3')

Saving model checkpoint to /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3
Configuration saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/config.json
Model weights saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/pytorch_model.bin
Feature extractor saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/preprocessor_config.json
tokenizer config file saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3/special_tokens_map.json


## Inference

In [ ]:
from PIL import Image
import torch
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = LayoutLMv3ForTokenClassification.from_pretrained("/content/drive/MyDrive/Colab Notebooks/Work/LayoutLMModel/AnesthesiaLayoutLMV3")
# processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)
# model.to(device)

In [ ]:
# choose an example from the eval set for inference (we can use a test set instead)
dataset = load_from_disk(f'/content/raw_data')
example = dataset["test"][7]
print(example.keys())

dict_keys(['id', 'tokens', 'bboxes', 'ner_tags', 'image'])


In [ ]:
image = example["image"]
words = example["tokens"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="pt",truncation=True, padding="max_length")
for k,v in encoding.items():
  # encoding[k] = v.to('cuda')
  print(k,v.shape)

input_ids torch.Size([1, 512])
attention_mask torch.Size([1, 512])
bbox torch.Size([1, 512, 4])
labels torch.Size([1, 512])
pixel_values torch.Size([1, 3, 224, 224])


In [ ]:
# Next, we do a forward pass. We use torch.no_grad() as we don't require gradient computation.
model.eval()
with torch.no_grad():
  outputs = model(**encoding)

/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:811: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


In [ ]:
# The model outputs logits of shape (batch_size, seq_len, num_labels).
logits = outputs.logits
predictions = logits.argmax(-1).squeeze().tolist()

In [ ]:
# ground truth labels to compare predictions with  them
labels = encoding.labels.squeeze().tolist()

In [ ]:
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

token_boxes = encoding.bbox.squeeze().tolist()
width, height = image.size

true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]
true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

In [ ]:
def random_color():
  return np.random.randint(0,255,3)
def iob_to_label(label):
    label = label[2:]
    if not label:
      return 'other'
    return label
label2color = {f'{iob_to_label(label).lower()}':(random_color()[0],random_color()[1],random_color()[2]) for label in label2id.keys()}

check the inference result

In [ ]:
from PIL import ImageDraw, ImageFont

image = image.convert('RGBA')
tmp = image.copy()
# draw = ImageDraw.Draw(tmp)
overlay = Image.new('RGBA', tmp.size,(0,0,0)+(0,))
draw = ImageDraw.Draw(overlay)
font = ImageFont.load_default()

# label2color = {'question':'blue', 'answer':'green', 'header':'orange', 'other':'violet'}

for prediction, box in zip(true_predictions, true_boxes):
    predicted_label = iob_to_label(prediction).lower()
    draw.rectangle(box, outline=label2color[predicted_label],width=3,fill=label2color[predicted_label]+(int(255*0.25),))
    draw.text((box[0] + 10, box[1] - 10), text=predicted_label, fill=label2color[predicted_label],font=font)

img = Image.alpha_composite(tmp, overlay)
img = img.convert("RGB")
img

compare with ground truth

In [ ]:
image = example["image"]
image = image.convert('RGBA')
tmp = image.copy()
# draw = ImageDraw.Draw(tmp)
overlay = Image.new('RGBA', tmp.size,(0,0,0)+(0,))
draw = ImageDraw.Draw(overlay)
font = ImageFont.load_default()

for word, box, label in zip(example['tokens'], example['bboxes'], example['ner_tags']):
  actual_label = iob_to_label(id2label[label]).lower()
  box = unnormalize_box(box, width, height)
  draw.rectangle(box, outline=label2color[actual_label],width=3,fill=label2color[actual_label]+(int(255*0.25),))
  draw.text((box[0] + 10, box[1] - 10), text=actual_label, fill=label2color[actual_label],font=font)

img = Image.alpha_composite(tmp, overlay)
img = img.convert("RGB")
img